In [34]:
import os
import requests
import py3Dmol
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign
from Bio.PDB import PDBParser, Superimposer, PDBIO
from Bio.PDB.cealign import CEAligner
from openbabel import pybel
import subprocess


ModuleNotFoundError: No module named 'PDB'

In [12]:
PDB_DIR = "pdb_files/"
LIGAND_DIR = "ligands/"
DOCKING_DIR = "docking_results/"
os.makedirs(PDB_DIR, exist_ok=True)
os.makedirs(LIGAND_DIR, exist_ok=True)
os.makedirs(DOCKING_DIR, exist_ok=True)



In [13]:
# --- 1. Fetch PDB Files ---

def fetch_pdb(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_path = f"{PDB_DIR}{pdb_id}.pdb"
    
    if response.status_code == 200:
        with open(pdb_path, 'wb') as f:
            f.write(response.content)
        print(f"{pdb_id} downloaded.")
        return pdb_path
    else:
        raise Exception(f"Failed to fetch PDB {pdb_id}")
    
# Fetch and prepare proteins
pdb_4iaq = fetch_pdb("4IAQ")
pdb_6g79 = fetch_pdb("6G79")

4IAQ downloaded.
6G79 downloaded.


In [18]:
# --- 2. Prepare Protein with PDBFixer ---

def prepare_protein(input_pdb, output_pdb):
    fixer = PDBFixer(filename=input_pdb)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    with open(output_pdb, 'w') as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f)
    
    print(f"Protein {input_pdb} prepared as {output_pdb}")
    
prepare_protein(pdb_4iaq, f"{PDB_DIR}4IAQ_prepared.pdb")
prepare_protein(pdb_6g79, f"{PDB_DIR}6G79_prepared.pdb")


Protein pdb_files/4IAQ.pdb prepared as pdb_files/4IAQ_prepared.pdb
Protein pdb_files/6G79.pdb prepared as pdb_files/6G79_prepared.pdb


In [24]:
# --- 3. Align Protein Structures ---

# def align_proteins(reference_pdb, target_pdb):
#     parser = PDB.PDBParser(QUIET=True)
#     ref_structure = parser.get_structure("ref", reference_pdb)
#     target_structure = parser.get_structure("target", target_pdb)
    
#     super_imposer = PDB.Superimposer()
#     ref_atoms = []
#     target_atoms = []
    
#     for ref_chain, target_chain in zip(ref_structure[0], target_structure[0]):
#         for ref_res, target_res in zip(ref_chain, target_chain):
#             if 'CA' in ref_res and 'CA' in target_res:
#                 ref_atoms.append(ref_res['CA'])
#                 target_atoms.append(target_res['CA'])
    
#     super_imposer.set_atoms(ref_atoms, target_atoms)
#     super_imposer.apply(target_structure.get_atoms())
    
#     aligned_pdb = target_pdb.replace(".pdb", "_aligned.pdb")
#     with open(aligned_pdb, 'w') as f:
#         io = PDB.PDBIO()
#         io.set_structure(target_structure)
#         io.save(f)
    
#     print(f"Aligned {target_pdb} to {reference_pdb}")
#     return aligned_pdb





def align_pdbs(reference_pdb, target_pdb):

    # Initialize PDB parser
    parser = PDBParser(QUIET=True)

    # Parse the structures
    ref_structure = parser.get_structure("ref", reference_pdb)
    target_structure = parser.get_structure("target", target_pdb)
    
    # Initialize CEAligner
    aligner = CEAligner()
    aligner.set_reference(ref_structure)
    
    # Perform CE alignment
    aligner.align(target_structure)

    # Save the aligned structures
    io = PDBIO()
    io.set_structure(ref_structure)
    io.save('pdb_files/4IAQ_aligned.pdb')

    io.set_structure(target_structure)
    io.save('pdb_files/6G79_aligned.pdb')
    
# Align proteins
aligned_6g79 = align_pdbs(f"{PDB_DIR}4IAQ_prepared.pdb", f"{PDB_DIR}6G79_prepared.pdb")

In [25]:
### Check the alignment

# Create 3Dmol.js viewer
v = py3Dmol.view(width=1000, height=800)

#Load protein structure
v.addModel(open('pdb_files/4IAQ_aligned.pdb').read())  # Protein 1
v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

v.addModel(open('pdb_files/6G79_aligned.pdb').read())  # Protein 2
v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

v.rotate(90)  # Rotate for better visualization
v.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [35]:
# --- 4. Select Binding Site ---

def select_binding_site(pdb_file, ligand_resname):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    ligand_atoms = []
    for model in structure:
        for chain in model:
            for res in chain:
                if res.resname == ligand_resname:
                    for atom in res:
                        ligand_atoms.append(atom.coord)
    
    if not ligand_atoms:
        raise ValueError(f"Ligand {ligand_resname} not found in {pdb_file}")

    # Calculate center of mass for the ligand binding site
    center = sum(ligand_atoms) / len(ligand_atoms)
    print(f"Binding site center for {pdb_file}: {center}")
    return center


In [36]:
# --- 5. Prepare Ligand from SMILES (.sdf format) ---

def prepare_ligand_from_smiles(smiles, ligand_name):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    ligand_path = f"{LIGAND_DIR}{ligand_name}.sdf"
    
    writer = Chem.SDWriter(ligand_path)
    writer.write(mol)
    writer.close()
    
    print(f"Ligand {ligand_name} saved as {ligand_path}")
    return ligand_path


In [37]:
# --- 6. Perform Docking with GNINA ---

import os
import subprocess

def perform_docking(protein_pdb, ligand_sdf, center, docking_dir="./docking_results/"):
    # Ensure docking directory exists
    if not os.path.exists(docking_dir):
        os.makedirs(docking_dir)
    
    output_sdf = f"{docking_dir}{os.path.basename(protein_pdb).split('.')[0]}_{os.path.basename(ligand_sdf).split('.')[0]}.sdf"
    
    command = [
        "gnina", "-r", protein_pdb, "-l", ligand_sdf,
        "--center_x", str(center[0]), "--center_y", str(center[1]), "--center_z", str(center[2]),
        "--size_x", "15", "--size_y", "15", "--size_z", "15",
        "-o", output_sdf, "--exhaustiveness", "8", "--seed 0"
    ]
    
    # Run the command and capture the result
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check if the command was successful
    if result.returncode == 0:
        print(f"Docking complete: {output_sdf}")
    else:
        print(f"Error during docking: {result.stderr}")
    
    return output_sdf


In [38]:
# --- 7. Calculate RMSD ---

def calculate_rmsd(ref_sdf, docked_sdf):
    ref_mol = Chem.SDMolSupplier(ref_sdf)[0]
    docked_mol = Chem.SDMolSupplier(docked_sdf)[0]
    
    rmsd = rdMolAlign.GetBestRMS(docked_mol, ref_mol)
    print(f"RMSD: {rmsd:.2f} Å")
    return rmsd

In [39]:
# --- 8. Display the results ---

### best mode

import py3Dmol

def show_best_3d(protein, ligand, docked):
    # Create 3Dmol.js viewer
    v = py3Dmol.view()

    # Load protein structure
    v.addModel(open(protein).read())  # Protein
    v.setStyle({'cartoon': {}, 'stick': {'radius': .1}})

    # Load ligand structure
    v.addModel(open(ligand).read())  # Ligand
    v.setStyle({'model': 1}, {'stick': {'colorscheme': 'dimgrayCarbon', 'radius': .125}})

    # Extract only the first docking pose from the SDF file
    with open(docked) as sdf_file:
        best_pose = []
        for line in sdf_file:
            if line.startswith('$$$$'):
                break  # End of the first molecule
            best_pose.append(line)

    # Add the best docking pose to the viewer
    v.addModel("".join(best_pose), 'sdf')  # Load only the best pose
    v.setStyle({'model': 2}, {'stick': {'colorscheme': 'greenCarbon'}})

    # Final adjustments
    v.zoomTo({'model': 1})  # Focus on the protein
    v.rotate(90)  # Rotate for better visualization
    v.show()


### all modes
def show_all_3d(protein, ligand, docked):
    v = py3Dmol.view()
    v.addModel(open(protein).read())
    v.setStyle({'cartoon':{},'stick':{'radius':.1}})
    v.addModel(open(ligand).read())
    v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
    v.addModelsAsFrames(open(docked).read())
    v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
    v.zoomTo({'model':1})
    v.rotate(90)
    v.animate({'interval':2000})


In [40]:
# --- Execute Workflow ---

import os

# Define directories for PDB and ligand files
PDB_DIR = "./pdb_files/"
LIGAND_DIR = "./ligand_files/"

# Create directories if they don't exist
for directory in [PDB_DIR, LIGAND_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)




# Align proteins
aligned_6g79 = align_pdbs(f"{PDB_DIR}4IAQ_prepared.pdb", f"{PDB_DIR}6G79_prepared.pdb")

# Prepare ligands
ligand_2gm = prepare_ligand_from_smiles("CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N[C@]5(C)O[C@@]6(O)[C@@H]7CCCN7C(=O)[C@H](Cc8ccccc8)N6C5=O", "2GM")  # Example SMILES
ligand_ep5 = prepare_ligand_from_smiles("[NH3+]CCc1c[nH]c2ccc(OCC(=O)N3CCN(CC3)c4ccc(cc4)C#N)cc12", "EP5")

# Binding site selection
center_4iaq = select_binding_site(f"{PDB_DIR}4IAQ_prepared.pdb", "2GM")

# Docking
docked_4iaq_2gm = perform_docking(f"{PDB_DIR}4IAQ_prepared.pdb", ligand_2gm, center_4iaq)

# RMSD Calculation
calculate_rmsd(ligand_2gm, docked_4iaq_2gm)


Ligand 2GM saved as ./ligand_files/2GM.sdf
Ligand EP5 saved as ./ligand_files/EP5.sdf


ValueError: Ligand 2GM not found in ./pdb_files/4IAQ_prepared.pdb